In [1]:
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
%pip install scikit-learn
!pip install numpy==1.23.5
!pip install gensim
nltk.download('punkt_tab')
nltk.download('stopwords')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.6 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 2.1.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, b

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### **1. Load the dataset into a DataFrame.**

---



In [2]:
df = pd.read_csv('isw_data_new.csv')

In [3]:
df

,Year,Month,Day,Bold Text
0,2022,2,28,russia deployed additional heavy forces artill...
1,2022,3,1,russian forces setting conditions envelop kyiv...
2,2022,3,2,russian forces resumed offensive operations ag...
3,2022,3,3,russian forces opened new line advance belarus...
4,2022,3,4,russian forces advanced rapidly eastern outski...
...,...,...,...,...
1075,2025,2,26,us president donald trump ukrainian president ...
1076,2025,2,27,russian president vladimir putin senior russia...
1077,2025,2,28,ukrainian president volodymyr zelensky us pres...
1078,2025,3,1,senior us officials suggesting united states m...


### **2. Clean the text data**

---


In [4]:
def clean_text(text):
    if pd.isnull(text):
        return ''

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'isw.?s interactive map of the russian invasion of ukraine.*?static maps present in this report', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Click.*?(?=[A-Z]|$)', '', text, flags=re.DOTALL)
    text = re.sub(r'^.*?\b(am|pm)\b\s*(et)?', '', text, flags=re.IGNORECASE)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove punctuation and extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()



df['Bold Text'] = df['Bold Text'].apply(clean_text)

def cut_before_pm(text):
     if isinstance(text, str):
         lowered = text.lower()
         index = lowered.find("pm")
         if index != -1:
             return text[index + 7:]
     return text
df["Bold Text"][0:14] = df["Bold Text"][0:14].apply(cut_before_pm)

def cut_before_pm_2(text):
     if isinstance(text, str):
         lowered = text.lower()
         index = lowered.find("pm")
         if index != -1:
             return text[index + 6:]
     return text
df["Bold Text"][14:] = df["Bold Text"][14:].apply(cut_before_pm_2)

<ipython-input-4-6ec130cbd72f>:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Bold Text"][0:14] = df["Bold Text"][0:14].apply(cut_before_pm)
<ipython-input-4-6ec130cbd72f>:27: SettingWithCopyWarning: 
A value is trying to be set on a 

### **3. Define a set of stop words to be removed from the text.**
---

In [5]:
stop_words = set([
    "a", "an", "the", "and", "or", "but", "if", "so", "then",
    "to", "of", "in", "on", "at", "by", "with", "from", "into", "over",
    "under","is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "do", "does", "did", "will", "would", "can",
    "could", "should", "may", "might", "must",
    "this", "that", "these", "those",
    "very", "too", "just", "only", "still",
    "some", "any", "every", "each",
    "what", "who", "where", "when", "why", "how"
])

def clean_text(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['Bold Text'] = df['Bold Text'].apply(clean_text)

###**The function processes a given text by tokenizing it, removing stop words, filtering out non-alphabetic characters, and then reducing the remaining words to their root forms using stemming.**

In [6]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def clean_and_stem(text):
    words = nltk.word_tokenize(text)
    filtered_words = [
        stemmer.stem(word)
        for word in words
        if word not in stop_words and word.isalpha()
    ]
    return ' '.join(filtered_words)

df['Bold Text'] = df['Bold Text'].apply(clean_and_stem)

### **4. Remove numbers.**
---

In [7]:
def remove_numbers(text):
        if isinstance(text, str):
          return re.sub(r'\\d+', '', text)
        return text

        df['Bold Text'] = df['Bold Text'].apply(remove_numbers)

### **5. Convert the cleaned text into TF-IDF features.**
---

In [8]:
vectorizer = TfidfVectorizer(ngram_range=(2, 2), max_features=1000)
X_tfidf = vectorizer.fit_transform(df['Bold Text'])
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.tail()


,access isw,across ukrain,action belaru,activ belaru,activ kherson,activ russianoccupi,administr border,administr control,advanc area,advanc avdiivka,...,within daili,within russia,within russian,within tube,without conduct,xi jinp,yevgeni prigozhin,zaporizhia oblast,zaporizhzhia nuclear,zelenski state
1075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015569,0.0,0.0,...,0.0,0.0,0.0,0.031025,0.016149,0.0,0.0,0.027246,0.0,0.0
1076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014143,0.0,0.0,...,0.0,0.0,0.0,0.028184,0.014671,0.0,0.0,0.024751,0.0,0.0
1077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013233,0.0,0.0,...,0.0,0.0,0.0,0.026370,0.013726,0.0,0.0,0.046316,0.0,0.0
1078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014534,0.0,0.0,...,0.0,0.0,0.0,0.028963,0.015076,0.0,0.0,0.000000,0.0,0.0
1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015612,0.0,0.0,...,0.0,0.0,0.0,0.031112,0.016194,0.0,0.0,0.027322,0.0,0.0


In [9]:
tfidf_df.to_pickle("tfidf_df.pkl")


### **6. Calculate the importance of each word based on TF-IDF scores.**
---

In [10]:
word_importance = np.sum(X_tfidf.toarray(), axis=0)
word_importance_df = pd.DataFrame({
    'word': vectorizer.get_feature_names_out(),
    'importance': word_importance
})

word_importance_df = word_importance_df.sort_values(by='importance', ascending=False)
word_importance_df.head(10)

,word,importance
749,russian object,139.362560
732,russian forc,99.718541
921,ukrainian forc,65.422493
243,donetsk oblast,52.432245
304,forc continu,50.336041
815,sourc use,46.760743
742,russian militari,42.364551
540,object captur,42.026087
588,offens oper,41.895441
567,oblast russian,39.343580


### **7. Calculate the mean vector for each article.**
---

In [11]:
article_vectors = X_tfidf.mean(axis=1)
article_vectors_df = pd.DataFrame(article_vectors, columns=[f'feat_{i}' for i in range(article_vectors.shape[1])])
article_vectors_df.head()

,feat_0
0,0.004729
1,0.003795
2,0.003004
3,0.004149
4,0.002797


In [12]:
df['text_vector'] = article_vectors_df.astype('float64').values

df[['text_vector']].head()

,text_vector
0,0.004729
1,0.003795
2,0.003004
3,0.004149
4,0.002797


In [13]:
df = df.drop('Bold Text', axis=1)
df

,Year,Month,Day,text_vector
0,2022,2,28,0.004729
1,2022,3,1,0.003795
2,2022,3,2,0.003004
3,2022,3,3,0.004149
4,2022,3,4,0.002797
...,...,...,...,...
1075,2025,2,26,0.008497
1076,2025,2,27,0.008557
1077,2025,2,28,0.008758
1078,2025,3,1,0.008391


In [14]:
df.loc[250:260]

,Year,Month,Day,text_vector
250,2022,11,8,0.010278
251,2022,11,9,0.009661
252,2022,11,10,0.009626
253,2022,11,11,0.009390
254,2022,11,12,0.009323
255,2022,11,13,0.002894
256,2022,11,14,0.009845
257,2022,11,15,0.001660
258,2022,11,16,0.009373
259,2022,11,17,0.009641


In [ ]:
df.to_csv('isw_data_without_num_stopword.csv', index=False)